### First attempt at Microsoft Azure AutoML

Lets use an experiment workspace I already created from previous deploymnet exercise. note that I saved my key in another file & import it

In [1]:
#import subscribtionId and necessary libraries

from azureml.core import Workspace,Experiment,Run
import pandas as pd
from my_config import key

ws = Workspace.get(name="AMLSWorkspace3",
               subscription_id=key,
               resource_group='rgAMLSLearnworkspace')

A dataset that contains some sample diabetes data: age, sex, body mass index, average blood pressure, and six blood serum measurements, as well a quantitative measure of disease progression. We want to predict progression after one year

In [2]:
# Load the diabetes dataset, a well-known built-in small dataset that comes with scikit-learn.
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

X, y = load_diabetes(return_X_y = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print('Done')

Done


#### Now, you'll create an experiment in this workspace by using the following code:

In [3]:
from azureml.core.experiment import Experiment

experiment = Experiment(workspace = ws, name = "my-autoML-experiment")

print('Done')

Done


#### Defining ML objectives & constraints
##### This allows you to configure your AutoML and some-what control the process

In [13]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
import logging

automl_config = AutoMLConfig(task = 'regression',
                  iteration_timeout_minutes = 40,
                  iterations = 9,
                  primary_metric = 'spearman_correlation',
                  n_cross_validations = 5,
                  debug_log = 'automl.log',
                  verbosity = logging.INFO,
                  X = X_train, 
                  y = y_train)

print('Done')

WARNING - The AutoMLConfig inputs you have specified will soon be deprecated. Please use the AutoMLConfig shown in our documentation: https://aka.ms/AutoMLConfig


Done


### Start the AutoML process

Notice that here, we do not specify an estimator I.e No configuration was specified for our run. The default is your local machine, hence, the next block runs on your local computer.

In [ ]:
### on your local machine

In [15]:
local_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_5e996a4a-1f0e-4f28-ac9f-c2839adc5b2e
Current status: DatasetCrossValidationSplit. Generating CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   StandardScalerWrapper RandomForest             0:01:05       0.6886    0.6886
         1   MinMaxScaler RandomForest                      0:01:07       0.6788    0.6886
         2   StandardScalerWrapper ExtremeRandomTrees   

#### To see our results in a better visual representation, we use the widget library

In [16]:
# monitor the run
from azureml.widgets import RunDetails

RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

We can easily retrieve the best model with this next block

In [17]:
#local_run.get_output gives us the model with the best score

best_run, fitted_model = local_run.get_output()

In [20]:
fitted_model

RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('prefittedsoftvotingregressor', PreFittedSoftVotingRegressor(estimators=[('5', RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('StandardScalerWrapper', <automl.client.core.runtime.model_wrappers.StandardScalerWrapper object at 0x7f45a05a8278>), ('LassoLars', LassoLars(alpha=0....4444444444444444, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.2222222222222222]))]),
          stddev=None)

#### If you want to train your model using an Azure ML virtual machine, visit the link below, an easy to follow documentation by microsoft

https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-auto-train-remote


In [26]:
#results
y_pred_train = fitted_model.predict(X_train)
y_residual_train = y_train - y_pred_train
y_pred_test = fitted_model.predict(X_test)
y_residual_test = y_test - y_pred_test

In [29]:
#print (y_residual_test)